## 201710957 comp. sci. 이유진
## Last Assignment

## 연습문제

주의:

 - 다른 과제보다 많이 약 10% 배점합니다

 - 다른 학생과 협력하지 마세요.

 - 제한시간이 다른 과제에 비해 짧아요 (3.5일)



A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명
-----|-----
setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.
setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.
play | 가위, 바위, 보의 승패를 정한다.
distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.
getMatchResult() | "A wins", "B wins", "tie" 결과를 출력



노드에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력

## 1) Solidity 코딩

In [1]:
%%writefile C:\Users\sofod\Code\201710957\src\lastAssignment.sol
pragma solidity ^0.5.0;

contract LastAssignment{
    
    mapping(address => uint) balanceOf;
    uint8 numA;
    uint8 numB;
    address owner;
    uint betting =0;
    uint8 winner;
    
    constructor() public {
        owner = msg.sender;
    }
    
    function deposit(uint amount) payable public{
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
    }

    function setA() public {
        require( balanceOf[msg.sender] >= 1000);
        numA = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3); //set rock , scissors or paper
        balanceOf[msg.sender] -= 1000;
        betting += 1000;
        
    }
  
    function setB(address payable bAddr, uint8 _num) payable public{
        require( balanceOf[bAddr] >= 1000);

        balanceOf[bAddr] -= 1000;
        betting += 1000;
        
        numB = _num;
        
    }
    function getmachResult() public view returns(string memory){
        
        if(winner == 2) //draw
            return "tie";
        
        if( winner == 1)
            return "B wins";
        
        if(winner == 0)
            return "A wins";
        
    }
    function getNumA() public view returns(uint8){
        return numA;
    }
    function getNumB() public view returns(uint8){
        return numB;
    }
    
    
    function distributeBetAmount( address payable bAddr) payable public{
        if(winner == 0){ // Winner is A
            balanceOf[owner] += betting;
            betting = 0;
        }
        else if( winner == 1){ //winner is B
            balanceOf[bAddr] += betting;
            betting = 0;
        }
        else{ // tie
            
        }
    }

    
  
    function play() public returns(uint8){
        
        if( numA == numB ){ //draw (0,0), (1,1) , (2,2)
            winner = 2;
            return winner;
        }
        else{
            //B가 이겼을때
            if( numB == (numA+1)%3){ // (0,1) (1,2) (2,0)
                winner = 1;
                return winner;
            }
            //A가 이겼을때 
            else{ //(0,2), (1,0), (2,0)
                winner = 0;
                return winner;
            }
        }
        
        
    }
    
    function getBalance(address addr) public view returns(uint){
        return balanceOf[addr];
    }

}

Overwriting C:\Users\sofod\Code\201710957\src\lastAssignment.sol


## 2) Solc를 이용하여 컴파일

In [2]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.0+commit.1d4f565a.Windows.msvc


In [3]:
!solc --abi --bin --gas C:\Users\sofod\Code\201710957\src\lastAssignment.sol


======= C:/Users/sofod/Code/201710957/src/lastAssignment.sol:LastAssignment =======
Gas estimation:
construction:
   25815 + 459000 = 484815
external:
   deposit(uint256):	20697
   distributeBetAmount(address):	26433
   getBalance(address):	761
   getNumA():	516
   getNumB():	610
   getmachResult():	infinite
   play():	21979
   setA():	61500
   setB(address,uint8):	61429
Binary: 
6080604052600060025534801561001557600080fd5b5033600160026101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506108f7806100666000396000f3fe608060405260043610610099576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806311be6b331461009e5780631fc376f7146100ef57806326b88351146101065780638782c0191461019657806393e84cd9146101c75780639bf5cdb2146101f8578063a1d7273514610229578063b6b55f251461026d578063f8b2cb4f1461029b575b600080fd5b6100ed600480360360408110156100b457600080fd5b81019080803573ffffffffffffffffffffffffffffffffffff

## 3) node 를 이용한 배포 (geth 사용)

In [4]:
!geth --exec "eth.coinbase" attach http://localhost:8445

"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3"


In [7]:
%%writefile C:\Users\sofod\Code\201710957\src\lastAssignment_deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abiArray=[{"constant":false,"inputs":[{"name":"bAddr","type":"address"},{"name":"_num","type":"uint8"}],"name":"setB","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getmachResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getNumA","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getNumB","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"bAddr","type":"address"}],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"addr","type":"address"}],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x"+"6080604052600060025534801561001557600080fd5b5033600160026101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506108f7806100666000396000f3fe608060405260043610610099576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806311be6b331461009e5780631fc376f7146100ef57806326b88351146101065780638782c0191461019657806393e84cd9146101c75780639bf5cdb2146101f8578063a1d7273514610229578063b6b55f251461026d578063f8b2cb4f1461029b575b600080fd5b6100ed600480360360408110156100b457600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190803560ff169060200190929190505050610300565b005b3480156100fb57600080fd5b506101046103cd565b005b34801561011257600080fd5b5061011b6104d7565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561015b578082015181840152602081019050610140565b50505050905090810190601f1680156101885780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b3480156101a257600080fd5b506101ab6105e5565b604051808260ff1660ff16815260200191505060405180910390f35b3480156101d357600080fd5b506101dc6105fc565b604051808260ff1660ff16815260200191505060405180910390f35b34801561020457600080fd5b5061020d6106fd565b604051808260ff1660ff16815260200191505060405180910390f35b61026b6004803603602081101561023f57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610713565b005b6102996004803603602081101561028357600080fd5b8101908080359060200190929190505050610826565b005b3480156102a757600080fd5b506102ea600480360360208110156102be57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610883565b6040518082815260200191505060405180910390f35b6103e86000808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015151561034f57600080fd5b6103e86000808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055506103e8600260008282540192505081905550806001806101000a81548160ff021916908360ff1602179055505050565b6103e86000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015151561041c57600080fd5b600342446040516020018083815260200182815260200192505050604051602081830303815290604052805190602001206001900481151561045a57fe5b06600160006101000a81548160ff021916908360ff1602179055506103e86000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055506103e8600260008282540192505081905550565b60606002600360009054906101000a900460ff1660ff161415610531576040805190810160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525090506105e2565b6001600360009054906101000a900460ff1660ff161415610589576040805190810160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090506105e2565b6000600360009054906101000a900460ff1660ff1614156105e1576040805190810160405280600681526020017f412077696e73000000000000000000000000000000000000000000000000000081525090506105e2565b5b90565b6000600160009054906101000a900460ff16905090565b60006001809054906101000a900460ff1660ff16600160009054906101000a900460ff1660ff16141561065c576002600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690506106fa565b600360018060009054906101000a900460ff160160ff1681151561067c57fe5b0660ff166001809054906101000a900460ff1660ff1614156106cb576001600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690506106fa565b6000600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690505b90565b60006001809054906101000a900460ff16905090565b6000600360009054906101000a900460ff1660ff1614156107ab57600254600080600160029054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506000600281905550610823565b6001600360009054906101000a900460ff1660ff161415610821576002546000808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506000600281905550610822565b5b5b50565b803414151561083457600080fd5b806000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b60008060008373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054905091905056fea165627a7a72305820b7ade630ea12695ccb2edcf23af148240e499031bdae6ecc8cc2a2f5b381c14c0029";

var mulContract = new web3.eth.Contract(_abiArray);
mulContract
    .deploy({
            data: _bin
    })
    .send({
     from: "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3",
     gas: '1000000'
    }, function (error, transactionHash){ 
            console.log("error :" + error);
            console.log("hash : " + transactionHash); 
    })
    .then(function(newContractInstance){
        console.log("address : " + newContractInstance.options.address)
    });

Overwriting C:\Users\sofod\Code\201710957\src\lastAssignment_deploy.js


In [8]:
!node C:\Users\sofod\Code\201710957\src\lastAssignment_deploy.js

error :null
hash : 0xf016184d67b5a2cd79de733e9da0f5cd3e9214f71ba05180ce66fa8aae547ecc
address : 0x378Cdda15862D85Be8ac2d1d9D34c2c7A2f3c4Ce


## 4) 사용

### async 방식을 이용하였다.
#### 가위 바위 보 는 순서대로 0,1,2 로 표현하였다.

In [9]:
%%writefile C:\Users\sofod\Code\201710957\src\lastAssignment_use.js

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));

var _abiArray=[{"constant":false,"inputs":[{"name":"bAddr","type":"address"},{"name":"_num","type":"uint8"}],"name":"setB","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getmachResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getNumA","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"play","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getNumB","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"bAddr","type":"address"}],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"addr","type":"address"}],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="0x"+"6080604052600060025534801561001557600080fd5b5033600160026101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506108f7806100666000396000f3fe608060405260043610610099576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806311be6b331461009e5780631fc376f7146100ef57806326b88351146101065780638782c0191461019657806393e84cd9146101c75780639bf5cdb2146101f8578063a1d7273514610229578063b6b55f251461026d578063f8b2cb4f1461029b575b600080fd5b6100ed600480360360408110156100b457600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190803560ff169060200190929190505050610300565b005b3480156100fb57600080fd5b506101046103cd565b005b34801561011257600080fd5b5061011b6104d7565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561015b578082015181840152602081019050610140565b50505050905090810190601f1680156101885780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b3480156101a257600080fd5b506101ab6105e5565b604051808260ff1660ff16815260200191505060405180910390f35b3480156101d357600080fd5b506101dc6105fc565b604051808260ff1660ff16815260200191505060405180910390f35b34801561020457600080fd5b5061020d6106fd565b604051808260ff1660ff16815260200191505060405180910390f35b61026b6004803603602081101561023f57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610713565b005b6102996004803603602081101561028357600080fd5b8101908080359060200190929190505050610826565b005b3480156102a757600080fd5b506102ea600480360360208110156102be57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610883565b6040518082815260200191505060405180910390f35b6103e86000808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015151561034f57600080fd5b6103e86000808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055506103e8600260008282540192505081905550806001806101000a81548160ff021916908360ff1602179055505050565b6103e86000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015151561041c57600080fd5b600342446040516020018083815260200182815260200192505050604051602081830303815290604052805190602001206001900481151561045a57fe5b06600160006101000a81548160ff021916908360ff1602179055506103e86000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055506103e8600260008282540192505081905550565b60606002600360009054906101000a900460ff1660ff161415610531576040805190810160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525090506105e2565b6001600360009054906101000a900460ff1660ff161415610589576040805190810160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090506105e2565b6000600360009054906101000a900460ff1660ff1614156105e1576040805190810160405280600681526020017f412077696e73000000000000000000000000000000000000000000000000000081525090506105e2565b5b90565b6000600160009054906101000a900460ff16905090565b60006001809054906101000a900460ff1660ff16600160009054906101000a900460ff1660ff16141561065c576002600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690506106fa565b600360018060009054906101000a900460ff160160ff1681151561067c57fe5b0660ff166001809054906101000a900460ff1660ff1614156106cb576001600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690506106fa565b6000600360006101000a81548160ff021916908360ff160217905550600360009054906101000a900460ff1690505b90565b60006001809054906101000a900460ff16905090565b6000600360009054906101000a900460ff1660ff1614156107ab57600254600080600160029054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506000600281905550610823565b6001600360009054906101000a900460ff1660ff161415610821576002546000808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055506000600281905550610822565b5b5b50565b803414151561083457600080fd5b806000803373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b60008060008373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054905091905056fea165627a7a72305820b7ade630ea12695ccb2edcf23af148240e499031bdae6ecc8cc2a2f5b381c14c0029";

var game = new web3.eth.Contract(_abiArray, '0x378Cdda15862D85Be8ac2d1d9D34c2c7A2f3c4Ce');


async function startGame(a) {
    
    
    const accounts = await web3.eth.getAccounts();
    
    //시작전 잔고없으면 충전시켜줌
    
    balA = await game.methods.getBalance(accounts[0]).call();
    balB = await game.methods.getBalance(accounts[1]).call();
    
    if(balA == 0){
        await game.methods.deposit(12345).send({from:accounts[0], gas : 77777 , value :12345});
        console.log("Deposit game money to PlayerA ") 
    }
    if(balB == 0){
        await game.methods.deposit(12345).send({from:accounts[1],  gas : 77777 , value :12345});
        console.log("Deposit game money to PlayerB ") 
    }
    
    
    balA = await game.methods.getBalance(accounts[0]).call();
    balB = await game.methods.getBalance(accounts[1]).call();
    console.log("Balance : " +balA +" , "+ balB);
    

    
    //본격 게임 시작
 
    console.log("---------------------------------------");
    console.log("Game Start!!");
    await game.methods.setA().send({from: accounts[0]});
    await game.methods.setB(accounts[1] , a).send({from:accounts[0], gas : 61500} );
    
    balA = await game.methods.getBalance(accounts[0]).call();
    balB = await game.methods.getBalance(accounts[1]).call();
    
    console.log("After betting : " +balA +" , "+ balB);
    
    numa = await game.methods.getNumA().call(); 
    numb = await game.methods.getNumB().call();
    console.log("A : " + numa + "  B : " + numb);

    
    await game.methods.play().send({from: accounts[0]}); //가위바위보 승패정하기
     
    

    await game.methods.getmachResult().call().then(console.log); 
    await game.methods.distributeBetAmount(accounts[1]).send({from: accounts[0]}); //베팅금액분배


    //게임후 잔고 출력
    balA =  await game.methods.getBalance(accounts[0]).call();
    balB =  await game.methods.getBalance(accounts[1]).call();
    console.log("After Game : " +balA +" , "+ balB);


    console.log("---------------------------------------");
 
    
}



startGame(2);
    
    


Overwriting C:\Users\sofod\Code\201710957\src\lastAssignment_use.js


####  첫 실행시에 mapping된 잔고가 0원이기때문에 게임을 시작할수 없어 0일경우 1234 wei를  Deposit 실행후에  시작하였다

In [10]:
!node C:\Users\sofod\Code\201710957\src\lastAssignment_use.js

Deposit game money to PlayerA 
Deposit game money to PlayerB 
Balance : 12345 , 12345
---------------------------------------
Game Start!!
After betting : 11345 , 11345
A : 0  B : 2
A wins
After Game : 13345 , 11345
---------------------------------------


다른 결과를 보기위해 여러번 실행해보았다.
Contract에 사용자들은 돈을 가지고있기때문에 Deposit은 일어나지않았다.

In [11]:
!node C:\Users\sofod\Code\201710957\src\lastAssignment_use.js

Balance : 13345 , 11345
---------------------------------------
Game Start!!
After betting : 12345 , 10345
A : 2  B : 2
tie
After Game : 12345 , 10345
---------------------------------------


In [12]:
!node C:\Users\sofod\Code\201710957\src\lastAssignment_use.js

Balance : 12345 , 10345
---------------------------------------
Game Start!!
After betting : 11345 , 9345
A : 2  B : 2
tie
After Game : 11345 , 9345
---------------------------------------


tie가 두번 일어났기때문에 betting 금액이 2000wei가  아닌 6000wei로 늘어나 게임이 끝난후 사용자A는 6000wei를 얻게 되었다.

In [13]:
!node C:\Users\sofod\Code\201710957\src\lastAssignment_use.js

Balance : 11345 , 9345
---------------------------------------
Game Start!!
After betting : 10345 , 8345
A : 0  B : 2
A wins
After Game : 16345 , 8345
---------------------------------------
